In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [2]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [3]:
fea_number=6
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A3_013.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A1_013', 'Input_A6_014', 'Input_A1_014', 'Input_A2_013',
       'Input_A6_013', 'Input_A5_014'], dtype=object)

## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,6)

train_rmse: 0.0013798295340172293
test_rmse: 0.0014260865268315675
test_r2: 0.24349209285691095


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,5e-06)

test_rmse_ave: 0.0014508749359871344
[0.0013068012180729913, 0.0014739764515110743, 0.0014213300502428667, 0.001236427000591078, 0.0012581921466134135, 0.0015507169204870906, 0.0014816229952249195, 0.0017012416972292613, 0.0014150304367976012, 0.001663410443101048]


test_r2_ave: 0.23720160159406442
[0.26783846570744907, 0.3812036596091042, 0.3588809813831213, 0.08914354740099883, 0.26011706970446613, 0.1476314713052419, 0.3317275826355379, 0.11119991033352095, 0.1902175163740768, 0.23405581148712706]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.008],0.01)

test_rmse_ave: 0.0014730232992410653
[0.0013233062301078369, 0.0015422951182855532, 0.0014638057497393874, 0.0011984609946683875, 0.0012734736978977578, 0.0015541200431389741, 0.0015358445604665583, 0.001707212143965393, 0.0014309604196231912, 0.001700754034517615]


test_r2_ave: 0.21799046662102955
[0.2492271453891981, 0.32251198929036873, 0.31998940171286017, 0.14422265282909152, 0.24203521840297526, 0.14388623903480846, 0.28192030826403425, 0.10495053947955135, 0.1718823175645363, 0.1992788542428715]


## SVR

In [7]:
model=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,6,model)

train_rmse: 0.001387491821286287
test_rmse: 0.001426646714372851
test_r2: 0.2438017820827147


## XGB

In [10]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 140, 
              max_depth = 5, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.5,
              reg_alpha =  0,
              reg_lambda = 0.3)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.0014483741255004012
[0.0012688569945529336, 0.001460151594855275, 0.0014669712755363998, 0.0012808344843643308, 0.0013215521229617431, 0.0015713031273388983, 0.0014516344880085292, 0.0016006777836983523, 0.001391639976473073, 0.0016701194072144777]


test_r2_ave: 0.23669647549425918
[0.30973921114122616, 0.39275696746127786, 0.31704513316583116, 0.02253998248412903, 0.18372280931390883, 0.12485038786700553, 0.3585058918239722, 0.21317185388538495, 0.2167676676974185, 0.22786485010243773]


## RandomForest

In [9]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 8,
            min_samples_split = 5,
            max_leaf_nodes = 6,
            min_samples_leaf =3,
            random_state = 42)  
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.0014338314479529083
[0.0012725183613401707, 0.00146586431405279, 0.001431254798285981, 0.0012997732096852457, 0.0012348847567379916, 0.0015745067032198175, 0.0014522421402315177, 0.0016356270813526598, 0.0013269572643435266, 0.0016446858502793826]


test_r2_ave: 0.25211089946560306
[0.3057498817267845, 0.3879960987359953, 0.3498962138392775, -0.006579638061269488, 0.28727510722273397, 0.1212782366484173, 0.35796872229531107, 0.17843744351054902, 0.2878840913807469, 0.25120283735748494]


In [17]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=5e-06)
elastic_mod=ElasticNet(alpha=[0.008], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 140, 
              max_depth = 5, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.5,
              reg_alpha =  0,
              reg_lambda = 0.3)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 8,
            min_samples_split = 5,
            max_leaf_nodes = 6,
            min_samples_leaf =3,
            random_state = 42)    
# vote_mod = VotingRegressor([ ('Lasso', lasso_mod),('SVR', svr_mod), ('Linear', lr), ('Elastic', elastic_mod),
#                             ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])

vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
                            ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.0014357506755862945
[0.0012867291598793074, 0.0014681563811964388, 0.0014123488071120668, 0.0012477015114304405, 0.0012293494424189856, 0.0015356049828625573, 0.0014729726172508957, 0.0016725013894750187, 0.001384850786391439, 0.0016472916778457968]


test_r2_ave: 0.25271696853267317
[0.29015727873565755, 0.3860807090052121, 0.3669577170039314, 0.07245629839902146, 0.29365030019822935, 0.16416340779641592, 0.33950814968081067, 0.14097653552270262, 0.22439110822152253, 0.248828180763228]


In [20]:

stack_mod = StackingRegressor(regressors=[lr, svr_mod,  vote_mod], 
                           meta_regressor=random_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.0014454790947563668
[0.001365265714287983, 0.0014390990439583178, 0.001423710745027116, 0.0013202925264938785, 0.001279539215573983, 0.001565365478078372, 0.0014025581433549828, 0.0016804930712938345, 0.001339433526829038, 0.001639033482666162]


test_r2_ave: 0.2360037870855379
[0.20086120343071545, 0.4101412644863669, 0.3567314637967772, -0.038611933612157756, 0.23479769548461948, 0.1314519320166595, 0.4011475029257118, 0.13274761111392175, 0.27443028667142577, 0.25634084454133865]


In [21]:
ave(skew_data[title],y,0.3,stack_mod,0.4,vote_mod,0.3,svr_mod)

test_rmse_ave: 0.001433327968509201
[0.001302110709991652, 0.001460513889333319, 0.0014071153768294338, 0.0012567457026195891, 0.001232488200716865, 0.0015280851777526284, 0.0014568008072048078, 0.0016717765263742385, 0.0013804419430914411, 0.0016372013511780357]


test_r2_ave: 0.25414867554953935
[0.273084935229038, 0.3924555904101291, 0.3716404760247273, 0.0589606238863396, 0.2900388107553562, 0.17232949064744763, 0.3539316538161257, 0.14172097673188877, 0.22932174022732277, 0.25800245776701813]
